### We will be learning how to create tables and linking them together, and powerful tricks for SQL with your python code

### Normalized Design

#### In a normalized database, the relationship among the tables match the relationships that are really there among the data

#### A bunch of theory behind this idea, see below

http://www.bkent.net/Doc/simple5.htm

#### Database normalization is the process of organizing the columns and tables of a relational database to reduce data redundancy and improve data integrity
#### It is also the process of simplifying the design of a database so that it achieves the optimum structure

https://en.wikipedia.org/wiki/Database_normalization

### Normalized Design

#### First rule: Every row has the same number of columns
#### Second rule: There is a unique key and everything in a row says something about the key
#### Third rule: Facts that don't relate to the key belong in different tables
#### Fourth rule: Tables shouldn't imply relationships that don't exist

### Create Table and Types

In [ ]:
# example below

create table tablename(
    column1 type[constraints],
    column2 type[constraints],
    ...);

#### Data types are different amongst all the SQL programs - POSTgreSQL, MySQL, SQlite

### Creating and Dropping

In [ ]:
# create a database for operation
create database name[options];

# opposite of create - removes database/table
drop database name[options];

### Primary Keys
#### a column or columns that uniquely identify what each row in a table is about

In [ ]:
# example of how to do this

# for single column, write 'primary key' after column's type 
create table students(
    id serial primary key,
    name text,
    birthdate date
);

# if you have a multi column primary key, write 'primary key' after all the columns

create table postal_places(
    postal_code text,
    country text,
    name text,
    primary key (postal_code, country)
);

#### if every try to write a primary key that exists in the database already, the database will signal an error

In [1]:
# roll back is an undo command that can help in these situations

### Declaring Relationships
#### Cannot add different data types into a column declared with a specific data type. However, if trying to add or match a row with a primary key that doesn't exist in another table, we can declare relationships to help the database figure out what to do.
#### References provide referential integrity - columns that are supposed to refer to each other are guaranteed to do so

In [ ]:
# example of creating a relationship

create table sales(
    sku text references products(sku),
    sale_date date,
    count integer);

# using 'references' above tells the database that a particular column should have a reference to another table
# you can specify the column of the table, if needed, like the parenthesis above

### Foreign Keys
#### A column or set of columns in one tble that uniquely identifies rows in another table. It's possible to have two or more foreign keys

In [ ]:
# example

# table named students with 'id' column as primary key
create table students (
    id serial primary key,
    name text);

# table named courses with 'id' column as primary key
create table courses(
    id text primary key
    name text);

# table named grades with 'student' column referencing students
# table's id column, and 'course' column referencing courses 'id'
create table grades (
    student integer references students (id),
    course text references courses(id),
    grade text);

# these references help create foreign keys

### Self Joins
#### There are times where we will want to join a table to itself

In [ ]:
select a.id, b.id, a.building, a.room
from residences as a, residences as b
where a.building = b.building
    and a.room = b.room
    and a.id > b.id
order by a.building, a.room;

### Counting for Nulls!
#### left joins & right joins

In [ ]:
# example that counts rows with 0's

select programs.name, count(bugs.filename) as num
       from programs left join bugs
         on programs.filename = bugs.filename
       group by programs.name
       order by num;

### What's a DB-API
#### Behind our server is Python Code that connects to SQLite, which is hooked up, via a web standard called a DB-API, to a database. If learn DB-API system, you can control database in Python libraries through any number of database systems. Python DB-API is a standard that allows to hook up to different databases. If learn DB-API functions, you can apply to any database system. Libraries include sqlite3, psycopg2, pyodbc, mysql.connector.

In [1]:
# To see how the various functions in the DB-API work, take a look at this code,
# then the results that it prints when you press "Test Run".
#
# Then modify this code so that the student records are fetched in sorted order
# by student's name.
#

import sqlite3

# Fetch some student records from the database.

## connects to the database 'students' using sqlite3 functions in python
db = sqlite3.connect("students")

## to use the database, we need to get a cursor object .cursor() 
## this allows python to pass SQL statements to the cursor object 
## to execute them. after executing commands, look to commit changes 
c = db.cursor()

## sql commands are done with double quotes
## .execute does the query instructed; however, does not return the query
query = "select name, id from students order by name;"
c.execute(query)

## must execute query in order to retrieve any queries
## fetchone() returns a single row
## fetchall() command will return all rows however
rows = c.fetchall()

# First, what data structure did we get?
print "Row data:"
print rows

# And let's loop over it too:
print
print "Student names:"
for row in rows:
  print "  ", row[0]

db.close()


OperationalError: no such table: students

### For more information about how to control sqlite in jupyter notebook, check out the link below.

http://www.bogotobogo.com/python/python_sqlite_connect_create_drop_table.php

http://www.bogotobogo.com/python/python_sqlite_select_update_delete.php

## Writing code with DB-API

### Every time you will use sqlite3, you will follow these commands

In [ ]:
# 1) importing database system library
import sqlite3
# 2) connect to database
# if using a database system over network, you may have to specify usernmae and password
# connection is good until you close it
conn = sqlite3.connect("Cookies")
# 3) cursor is used to scan through the results
# called cursor because you use it to scan through all data
cursor = conn.cursor()
# 4) execute a query through the cursor
cursor.execute(
    "select host_key from cookies limit 10")
# 5) if doing insert or any query that changes the database, you would have to commit it
results = cursor.fetchall()

print results
# when done, close connections. stale connections take resources.
conn.close()

## Inserts in DB-API
### must submit a commit after creating a change

In [ ]:
import sqlite3

db = sqlite3.connect("testdb")
c = db.cursor()
c.execute("insert into balloons values ('blue', 'water') ")
db.commit()
db.close()

### changes in jupyter notebook happen in atomicity (either a transaction happens as a whole or not at all)

## Subqueries
### instead of selecting tables or joins of tables, you can also select resulting tables of selects in a query - this is called subqueries or subselecting. The result of a query is always a table, which is why you can have a select in a select 

In [ ]:
# query to find average score of the highest score
# SQL makes the subquery table have a name (this is the 'as maxes' in query)
'select avg(bigscore) from (select max(score) as bigscore from mooseball group by team) as maxes;''

In [ ]:
# Find the players whose weight is less than the average.
# 
# The function below performs two database queries in order to find the right players.
# Refactor this code so that it performs only one query.
#

def lightweights(cursor):
    """Returns a list of the players in the db whose weight is less than the average."""
    cursor.execute("select avg(weight) as av from players;")
    av = cursor.fetchall()[0][0]  # first column of first (and only) row
    cursor.execute("select name, weight from players where weight < " + str(av))
    return cursor.fetchall()

# turns into

def lightweights(cursor):
    """Returns a list of the players in the db whose weight is less than the average."""
    cursor.execute("select name, weight 
                   from players,
                   (select avg(weight) as av from players) as subq
                   where weight < av")
    return cursor.fetchall()

## Views
### a view is a select query stored in the database in a way that lets you use it like a table

In [ ]:
create view course_size as
    select course_id, count(*) as num
    from enrollment
    group by course_id;

## Working with CSVs

In [ ]:
# Exporting data to CSV in shell

sqlite> .mode csv
sqlite> .output newFile.csv
sqlite> SELECT * FROM myTable;
sqlite> .exit

# Importing data from CSV in shell

$ sqlite3 new.db   <--- "If you'd like your csv's in a new database remember to make it first."

sqlite> CREATE TABLE myTable() <--- Build your schema!
sqlite> .mode csv
sqlite> .import newFile.csv myTable

## Learned database design basics. Building normalized tables to represent various relationships among data. And you've spent time with POSTgreSQL and Python DB-API documentation - use them in your future work.

### Quiz 1 - U

### Quiz 2 - D

### Quiz 3 - A

### Quiz 4 - C

In [ ]:
##  After all the success promoting your music tour last section, 
##  a new friend has asked to partner up and build your own music website!
##  You'll need to rebuild your own database and import the data to your new system.

##  Let's first take a closer look at how to build and populate your local database.
##  The box below shows the Album table schema including Primary and Foreign Keys.
##  Have a look at this table and the CREATE TABLE statement below to see how they 
##  relate.

'''
First, disconnect from your Chinook database.
> .exit

Create a new database named whatever you'd like your store to be called.
$ sqlite3 UdaciousMusic.db

Now we can populate this database with our first table.

Here's a graphic showing some information about the Album table.
We can use this to build a table in our new database.

######################################################################
#                         Table: Album                               #
######################################################################
+--------------------+---------------+-----------------+--------------+
|      Columns       |   Data Type   |    Primary Key  |  Foreign Key |
+====================+===============+=================+==============+ 
|      AlbumId           INTEGER            YES              NO       |
|      Title             TEXT               NO               NO       |
|      ArtistId          INTEGER            NO               YES      |
|      UnitPrice         REAL               NO               NO       |
|      Quantity          INTEGER            NO               NO       |
+====================+===============+=================+==============+ 

We can use this information to decide how our schema should look.
Do you see how the schema below reflects the table above?

CREATE TABLE Album
(
    AlbumId INTEGER PRIMARY KEY,
    Title TEXT,
    ArtistId INTEGER,
    FOREIGN KEY (ArtistId) REFERENCES Artist (ArtistId) 
);

Try pasting the schema into your local database.  
Let's check to see if anything happened.

sqlite> .tables
Album <--- Do you see the Album table?  I hope so!

Now, do we have any data in our new table?

sqlite> SELECT * FROM Album;

Do you see data?  I hope not, we haven't added any yet!

Open the Album.sql tab.  You can copy and paste these lines directly into 
your sqlite terminal. (Use Ctrl+A or Command+A to select all lines when the
code editor is selected to select all the lines at once.)

Now try to run your query again.  You've got data... NICE!

'''
##  Use the previous example to help you construct the InvoiceLine table.
##  When you're ready, run you query to CREATE and populate the InvoiceLine table 
##  using data from the InvoiceLine.sql file. 

QUERY='''
CREATE TABLE InvoiceLine
(
InvoiceLineId INTEGER PRIMARY KEY,
InvoiceId INTEGER,
TrackId INTEGER,
UnitPrice REAL,
Quantity INTEGER,
FOREIGN KEY (InvoiceId) REFERENCES Invoice (InvoiceId),
FOREIGN KEY (TrackId) REFERENCES Track (TrackId)
);

'''

'''
######################################################################
#                         Table: InvoiceLine                         #
######################################################################
+--------------------+---------------+-----------------+--------------+
|      Columns       |   Data Type   |    Primary Key  |  Foreign Key |
+====================+===============+=================+==============+ 
|      InvoiceLineId     INTEGER            YES              NO       |
|      InvoiceId         INTEGER            NO               YES      |
|      TrackId           INTEGER            NO               YES      |
|      UnitPrice         REAL               NO               NO       |
|      Quantity          INTEGER            NO               NO       |
+====================+===============+=================+==============+ 
'''

##  These examples should help you build any remaining tables from the Chinook database.

##  Use .schema when connected to the Chinook database to see the tables and columns 
##  in your Chinook database, and try recreating them in your new database.
##  You can populate these tables using the sql files from the Downloadables section, or 
##  with CSV files, which you'll learn to do next!


### Quiz 5 - I (uploading CSVs to database)

In [ ]:
.mode csv
.import Album.csv Album

# REALLY IMPORTANT
## How to use sqlite3 in jupyter notebook and push database into a Pandas Dataframe!

In [4]:
##  Here's a playground to help you get a little more comfortable using DB API
##  Nothing here is graded, you've done a ton of work so far and I think 
##  it'd be great to relax and play a bit!

##  Connect to the Chinook database and try a few of your own ideas.
##  This is also a great place to try experiments building queries 
##  for the next few quizzes!

import sqlite3

# Fetch records from either chinook.db
db = sqlite3.connect("/Users/marcmaybaum/Desktop/Education/Udacity/Data_Analyst_Nanodegree/Project_3/chinook_db/chinook.db")
c = db.cursor()
QUERY = "SELECT * FROM Invoice;"
c.execute(QUERY)
rows = c.fetchall()

'''Uncomment to see your query in python'''
# print ("Row data:")
# print (rows)

'''Uncomment to print your query by row'''
# print ("your output:")
# for row in rows:
#  print ("  ", row[0:])

'''Uncomment to see your query as a pandas dataframe.
This is similar to the output you've been seeing throughout this course
You can learn more about pandas dataframes in our Intro to Data Analysis course!'''

import pandas as pd    
df = pd.DataFrame(rows)
print (df)

db.close()

       0   1                    2                                         3  \
0      1   2  2009-01-01 00:00:00                   Theodor-Heuss-Straße 34   
1      2   4  2009-01-02 00:00:00                          Ullevålsveien 14   
2      3   8  2009-01-03 00:00:00                           Grétrystraat 63   
3      4  14  2009-01-06 00:00:00                            8210 111 ST NW   
4      5  23  2009-01-11 00:00:00                           69 Salem Street   
5      6  37  2009-01-19 00:00:00                          Berger Straße 10   
6      7  38  2009-02-01 00:00:00                       Barbarossastraße 19   
7      8  40  2009-02-01 00:00:00                            8, Rue Hanovre   
8      9  42  2009-02-02 00:00:00                    9, Place Louis Barthou   
9     10  46  2009-02-03 00:00:00                          3 Chatham Street   
10    11  52  2009-02-06 00:00:00                         202 Hoxton Street   
11    12   2  2009-02-11 00:00:00                   

### Quiz 6 - O

In [14]:
db = sqlite3.connect("/Users/marcmaybaum/Desktop/Education/Udacity/Data_Analyst_Nanodegree/Project_3/chinook_db/chinook.db")
c = db.cursor()
QUERY = '''
SELECT Track.TrackId
FROM Track
JOIN Genre ON Genre.GenreId = Track.GenreId
JOIN MediaType ON MediaType.MediaTypeId = Track.MediaTypeId
WHERE Genre.Name = 'Pop' AND MediaType.Name = 'MPEG audio file'
GROUP BY TrackId;
'''
c.execute(QUERY)
rows = c.fetchall()

'''Uncomment to see your query in python'''
# print ("Row data:")
# print (rows)

'''Uncomment to print your query by row'''
# print ("your output:")
# for row in rows:
#  print ("  ", row[0:])

'''Uncomment to see your query as a pandas dataframe.
This is similar to the output you've been seeing throughout this course
You can learn more about pandas dataframes in our Intro to Data Analysis course!'''

import pandas as pd    
df = pd.DataFrame(rows)
print (df)

db.close()

      0
0   323
1   324
2   325
3   326
4   327
5   328
6   329
7   330
8   331
9   332
10  333
11  334
12  335
13  336


### Quiz 7 - U

In [15]:
db = sqlite3.connect("/Users/marcmaybaum/Desktop/Education/Udacity/Data_Analyst_Nanodegree/Project_3/chinook_db/chinook.db")
c = db.cursor()
QUERY = '''
SELECT Invoice.CustomerId
FROM Invoice
JOIN InvoiceLine ON Invoice.InvoiceId = InvoiceLine.InvoiceId
JOIN Track ON InvoiceLine.TrackId = Track.TrackId
JOIN Genre ON Track.GenreId = Genre.GenreId
WHERE Genre.Name = 'Jazz'
GROUP BY Invoice.CustomerId;
'''
c.execute(QUERY)
rows = c.fetchall()

'''Uncomment to see your query in python'''
# print ("Row data:")
# print (rows)

'''Uncomment to print your query by row'''
# print ("your output:")
# for row in rows:
#  print ("  ", row[0:])

'''Uncomment to see your query as a pandas dataframe.
This is similar to the output you've been seeing throughout this course
You can learn more about pandas dataframes in our Intro to Data Analysis course!'''

import pandas as pd    
df = pd.DataFrame(rows)
print (df)

db.close()

     0
0    3
1    5
2    7
3   14
4   16
5   17
6   18
7   19
8   20
9   21
10  22
11  23
12  30
13  31
14  32
15  35
16  37
17  38
18  39
19  40
20  42
21  43
22  44
23  46
24  49
25  50
26  51
27  53
28  54
29  56
30  58
31  59


### Quiz 8 - 

In [20]:
db = sqlite3.connect("/Users/marcmaybaum/Desktop/Education/Udacity/Data_Analyst_Nanodegree/Project_3/chinook_db/chinook.db")
c = db.cursor()
QUERY = '''
SELECT Genre.Name, COUNT(Track.TrackId) as Num
FROM Genre, (SELECT AVG(Track.Milliseconds) as AvgLength FROM Track) as subq
JOIN Track ON Genre.GenreId = Track.GenreId
WHERE Track.Milliseconds < AvgLength
GROUP BY Genre.Name
ORDER BY Num desc;
'''
c.execute(QUERY)
rows = c.fetchall()

'''Uncomment to see your query in python'''
# print ("Row data:")
# print (rows)

'''Uncomment to print your query by row'''
# print ("your output:")
# for row in rows:
#  print ("  ", row[0:])

'''Uncomment to see your query as a pandas dataframe.
This is similar to the output you've been seeing throughout this course
You can learn more about pandas dataframes in our Intro to Data Analysis course!'''

import pandas as pd    
df = pd.DataFrame(rows)
print (df)

db.close()

                     0     1
0                 Rock  1162
1                Latin   569
2   Alternative & Punk   320
3                Metal   300
4                 Jazz   116
5                Blues    72
6            Classical    60
7             R&B/Soul    60
8               Reggae    58
9                  Pop    46
10          Soundtrack    43
11         Alternative    38
12         Hip Hop/Rap    34
13   Electronica/Dance    28
14               World    28
15      Easy Listening    24
16         Heavy Metal    23
17          Bossa Nova    14
18       Rock And Roll    12
19               Drama     1
20               Opera     1
